In [337]:

## importing library
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from IPython.display import FileLinks
import warnings
warnings.filterwarnings('ignore')
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from tpot import TPOTRegressor
from tpot.builtins import StackingEstimator
from sklearn.linear_model import *
from sklearn.neighbors import KNeighborsRegressor

In [338]:
##cross validation to ensure that model doen't overfit
def cross(model, data, y, n_folds=10):
    y_pred = np.array([0]*len(df))
    kf = KFold(n_splits=n_folds)
    ii=0
    for train_idx, test_idx in kf.split(data):
        train = data.iloc[train_idx, :]
        test = data.iloc[test_idx, :]
        ytn = y[train_idx]
        yts = y[test_idx]
        model.fit(train, ytn)
        y_pred[test_idx] = model.predict(test).astype(int)
#         print(ii)
        ii+=1
    return y_pred

In [339]:
data = pd.read_csv('/kaggle/input/openiit/Insurance_Marketing-Customer-Value-Analysis (1).csv')

In [340]:
data.head()

,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Income,...,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size
0,BU79786,Washington,2763.519279,No,Basic,Bachelor,2/24/2011,Employed,F,56274,...,5,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize
1,QZ44356,Arizona,6979.535903,No,Extended,Bachelor,1/31/2011,Unemployed,F,0,...,42,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize
2,AI49188,Nevada,12887.431650,No,Premium,Bachelor,2/19/2011,Employed,F,48767,...,38,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize
3,WW63253,California,7645.861827,No,Basic,Bachelor,1/20/2011,Unemployed,M,0,...,65,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize
4,HB64268,Washington,2813.692575,No,Basic,Bachelor,2/3/2011,Employed,M,43836,...,44,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize


In [341]:
#########feature engineering of some usefull features
data['Effective To Date'] = pd.to_datetime(data['Effective To Date'])
data.head()
wee = [0]*data.shape[0]
for i in range(len(data)):
    wee[i] = data['Effective To Date'][i].year
data['year'] = wee

wee = [0]*data.shape[0]
for i in range(len(data)):
    wee[i] = data['Effective To Date'][i].month
data['month'] = wee

wee = [0]*data.shape[0]
for i in range(len(data)):
    wee[i] = data['Effective To Date'][i].day
data['day'] = wee

data['total_days'] = (data['year']-2010)*365+data['month']*30+data['day']
# data = data.drop(['day', 'month', 'year'], axis=1)
data.head()

,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Income,...,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,year,month,day,total_days
0,BU79786,Washington,2763.519279,No,Basic,Bachelor,2011-02-24,Employed,F,56274,...,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize,2011,2,24,449
1,QZ44356,Arizona,6979.535903,No,Extended,Bachelor,2011-01-31,Unemployed,F,0,...,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize,2011,1,31,426
2,AI49188,Nevada,12887.431650,No,Premium,Bachelor,2011-02-19,Employed,F,48767,...,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize,2011,2,19,444
3,WW63253,California,7645.861827,No,Basic,Bachelor,2011-01-20,Unemployed,M,0,...,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize,2011,1,20,415
4,HB64268,Washington,2813.692575,No,Basic,Bachelor,2011-02-03,Employed,M,43836,...,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize,2011,2,3,428


In [342]:
drop = ['Customer', 'Customer Lifetime Value', 'Effective To Date']
y = data['Customer Lifetime Value']
data = data.drop(drop, axis=1)

In [343]:
cata=[]
for i  in data.columns:
    print(i, data[i].dtype)
    if data[i].dtype == 'object':
        cata.append(i)

State object
Response object
Coverage object
Education object
EmploymentStatus object
Gender object
Income int64
Location Code object
Marital Status object
Monthly Premium Auto int64
Months Since Last Claim int64
Months Since Policy Inception int64
Number of Open Complaints int64
Number of Policies int64
Policy Type object
Policy object
Renew Offer Type object
Sales Channel object
Total Claim Amount float64
Vehicle Class object
Vehicle Size object
year int64
month int64
day int64
total_days int64


In [344]:
### processing categorical data
from sklearn.preprocessing import LabelEncoder

for i in cata:
    le = LabelEncoder()
    data[i] = le.fit_transform(data[i])
    
data.head()

,State,Response,Coverage,Education,EmploymentStatus,Gender,Income,Location Code,Marital Status,Monthly Premium Auto,...,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,year,month,day,total_days
0,4,0,0,0,1,0,56274,1,1,69,...,2,0,0,384.811147,5,1,2011,2,24,449
1,0,0,1,0,4,0,0,1,2,94,...,5,2,0,1131.464935,0,1,2011,1,31,426
2,2,0,2,0,1,0,48767,1,1,108,...,5,0,0,566.472247,5,1,2011,2,19,444
3,1,0,0,0,4,1,0,1,1,106,...,1,0,2,529.881344,3,1,2011,1,20,415
4,4,0,0,0,1,1,43836,0,2,73,...,3,0,0,138.130879,0,1,2011,2,3,428


In [345]:
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
cat = LGBMRegressor()
lg=cat

In [346]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import *#mean_squared_error
from sklearn.ensemble import *
# scores = cross_val_score(clf, iris.data, iris.target, cv=5)


In [347]:
lg = RandomForestRegressor(bootstrap=True, max_features=0.7500000000000001, min_samples_leaf=11, min_samples_split=9, n_estimators=100)
# valid = cross_val_score(lg, data, y, scoring='neg_mean_squared_error')
# np.sqrt(valid*-1).mean()

In [348]:
####### handling continous features
conti = list(data.columns)
for i in cata:
    conti.remove(i)

In [349]:
conti

['Income',
 'Monthly Premium Auto',
 'Months Since Last Claim',
 'Months Since Policy Inception',
 'Number of Open Complaints',
 'Number of Policies',
 'Total Claim Amount',
 'year',
 'month',
 'day',
 'total_days']

In [350]:
dum = pd.DataFrame(columns=[], index=data.index)
for i in cata:
    dum = pd.concat([dum, pd.get_dummies(data[i])], axis=1)
dum.columns = range(500, 500+dum.shape[1])
dum.head()

,500,501,502,503,504,505,506,507,508,509,...,547,548,549,550,551,552,553,554,555,556
0,0,0,0,0,1,1,0,1,0,0,...,0,0,0,0,0,0,1,0,1,0
1,1,0,0,0,0,1,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0
2,0,0,1,0,0,1,0,0,0,1,...,0,0,0,0,0,0,1,0,1,0
3,0,1,0,0,0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
4,0,0,0,0,1,1,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0


In [351]:
df = pd.concat([dum, data[conti]], axis=1)
# df['days'] = data['total_days']
# conti.append('days')

df.head()

,500,501,502,503,504,505,506,507,508,509,...,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Total Claim Amount,year,month,day,total_days
0,0,0,0,0,1,1,0,1,0,0,...,69,32,5,0,1,384.811147,2011,2,24,449
1,1,0,0,0,0,1,0,0,1,0,...,94,13,42,0,8,1131.464935,2011,1,31,426
2,0,0,1,0,0,1,0,0,0,1,...,108,18,38,0,2,566.472247,2011,2,19,444
3,0,1,0,0,0,1,0,1,0,0,...,106,18,65,0,7,529.881344,2011,1,20,415
4,0,0,0,0,1,1,0,1,0,0,...,73,12,44,0,1,138.130879,2011,2,3,428


In [352]:
# conti.append('days')
# conti.remove('total_days')

In [353]:
df.head()

,500,501,502,503,504,505,506,507,508,509,...,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Total Claim Amount,year,month,day,total_days
0,0,0,0,0,1,1,0,1,0,0,...,69,32,5,0,1,384.811147,2011,2,24,449
1,1,0,0,0,0,1,0,0,1,0,...,94,13,42,0,8,1131.464935,2011,1,31,426
2,0,0,1,0,0,1,0,0,0,1,...,108,18,38,0,2,566.472247,2011,2,19,444
3,0,1,0,0,0,1,0,1,0,0,...,106,18,65,0,7,529.881344,2011,1,20,415
4,0,0,0,0,1,1,0,1,0,0,...,73,12,44,0,1,138.130879,2011,2,3,428


In [354]:
from sklearn.preprocessing import StandardScaler
for i in conti:
    sc = StandardScaler()
    df[i] = sc.fit_transform(np.array(df[i]).reshape(-1, 1))
    
df.head()

,500,501,502,503,504,505,506,507,508,509,...,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Total Claim Amount,year,month,day,total_days
0,0,0,0,0,1,1,0,1,0,0,...,-0.703925,1.678099,-1.543287,-0.42225,-0.822648,-0.169640,0.0,1.075304,0.986776,1.491194
1,1,0,0,0,0,1,0,0,1,0,...,0.022691,-0.208186,-0.217334,-0.42225,2.106160,2.400737,0.0,-0.929969,1.795716,0.098461
2,0,0,1,0,0,1,0,0,0,1,...,0.429596,0.288205,-0.360680,-0.42225,-0.404247,0.455734,0.0,1.075304,0.408962,1.188426
3,0,1,0,0,0,1,0,1,0,0,...,0.371467,0.288205,0.606907,-0.42225,1.687759,0.329769,0.0,-0.929969,0.524524,-0.567629
4,0,0,0,0,1,1,0,1,0,0,...,-0.587666,-0.307465,-0.145661,-0.42225,-0.822648,-1.018843,0.0,1.075304,-1.440045,0.219568


In [355]:
from sklearn.model_selection import *
train , test, y_train, y_test = train_test_split(df, y, train_size=0.75)

In [356]:
from sklearn.linear_model import *
lr = LinearRegression()

In [357]:
train_copy =train.copy() 

In [358]:
lg

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features=0.7500000000000001, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=11, min_samples_split=9,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [359]:
lr = LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)

In [360]:
from sklearn.cluster import KMeans, AgglomerativeClustering, AffinityPropagation, SpectralClustering
SpectralClustering()

SpectralClustering(affinity='rbf', assign_labels='kmeans', coef0=1, degree=3,
                   eigen_solver=None, eigen_tol=0.0, gamma=1.0,
                   kernel_params=None, n_clusters=8, n_init=10, n_jobs=None,
                   n_neighbors=10, random_state=None)

In [361]:
#### kmeans clustrering of datapoints based on

# inertia = []

# clust = pd.DataFrame(columns=['no of clust', 'r2', 'mape'], index = range(20))
# X = df.drop('cluster', axis=1)
# from sklearn.cluster import KMeans
# from sklearn.cluster import KMeans, AgglomerativeClustering, AffinityPropagation, SpectralClustering
# i=0
# for k in range(1, 20):
#     print(k)
#     kmeans = KMeans(n_clusters=k, random_state=1, n_jobs=-1).fit(X)
    
#     df['cluster'] = SpectralClustering(n_clusters=k, n_jobs=-1).fit_predict(X)
#     y_pred = cross(lr, df, y)
#     print(r2_score(y, y_pred), "           ", mean_absolute_error(y, y_pred))
#     clust.iloc[i, 0] = k
#     clust.iloc[i, 1] = r2_score(y, y_pred)
#     clust.iloc[i, 2] = mean_absolute_error(y, y_pred)
#     i+=1

In [362]:
kmeans = KMeans(n_clusters=5, random_state=42, n_jobs=-1)
X = df.copy()
df['cluster'] = kmeans.fit_predict(df)

In [363]:
df.to_csv('df_openiit.csv', index=False)
FileLinks('.')

./
  perfomance_with_stack.csv
  perfomance.csv
  savs.csv
  __notebook_source__.ipynb
  df_openiit.csv
  train_open.csv
  open.csv
./catboost_info/
  catboost_training.json
  learn_error.tsv
  time_left.tsv
./catboost_info/learn/
  events.out.tfevents

In [364]:
lg = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features=0.7500000000000001, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=11, min_samples_split=9,
                      min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                      oob_score=False, random_state=None, verbose=0,
                      warm_start=False)

SyntaxError: invalid syntax (<ipython-input-364-cc80933c062d>, line 1)

In [ ]:
lr = lg

In [ ]:
# # Backward feature selection algorithm

# cols = df.columns
# model = lr
# # forest_scores = cross_val_score(model, df, y,
# #                                 scoring="neg_mean_squared_error", cv=10)

# train = df.copy()

# # origina = np.sqrt(-forest_scores).mean()

# lr = lg
# y_pred = cross(lr, train, y)

# origina = mean_absolute_error(y, y_pred)

# for i in cols:
   
# #     scores = [0]*train.columns
#     col = train.columns
   
#     for i in col:
#         c = train[i]

#         train = train.drop(i, axis=1)



#         y_pred = cross(lr, train, y)

#         new_error = mean_absolute_error(y, y_pred)
#         r2 = r2_score(y, y_pred)

#         print("ori", origina, "new", new_error, "r2", r2)

        
#         if(new_error < origina):
#             origina = new_error
#             column_to_remove = i
#         train[i] = c
#         print("done", i)
   
#     try:
#         print("--> removing", column_to_remove)
#         print(new_error)
#         train = train.drop(column_to_remove, axis=1)
#     except:
#         break
# to take this trunketed dataset as new use following line
# # df = train

In [ ]:
from sklearn.utils.testing import all_estimators
estimators = all_estimators()
est=[]
i=0
for name, class_ in estimators:
    if ~hasattr(class_, 'predict_proba'):
        est.append(class_)
        print(i, name)
    i+=1


In [ ]:
lis = [ 2, 7, 23, 27, 33, 35, 45, 49, 57, 58, 68, 87, 107, 136, 149, 151, 161, 184, 185, ]
est = []
for i in lis:
    est.append(estimators[i][1])
est

In [ ]:
####removed columns is available in train



train.to_csv('train_open.csv', index=False)
from IPython.display import FileLinks
FileLinks('.')

In [ ]:
from sklearn.model_selection import StratifiedKFold
def Stacking(model,train,y,test,n_fold=5):
#     train=pd.DataFrame(train)
    folds=KFold(n_splits=n_fold, random_state=None, shuffle=False)
    test_pred=np.empty((test.shape[0],1),'float32')
    train_pred=np.empty((0,1),'float32')
    for train_indices,val_indices in folds.split(train):
        x_train,x_val=train.iloc[train_indices, :],train.iloc[val_indices, :]
        y_train,y_val=y.iloc[train_indices],y.iloc[val_indices]
        model.fit(X=x_train,y=y_train)
        train_pred=np.append(train_pred,model.predict(x_val))
        test_pred=np.append(test_pred,model.predict(test))
    return pd.DataFrame(test_pred.reshape(-1,1)),pd.DataFrame(train_pred)

In [ ]:
# tests[i] ,trains[i]=Stacking(model=m,n_fold=10, train=df,test=test,y=y)

In [ ]:
from sklearn.metrics import *

In [ ]:
colss = [0]
trains = pd.DataFrame(columns=colss, index=range(len(df)))
tests = pd.DataFrame(columns=colss, index=range(len(test)))

trains.head()

In [ ]:
# trains = pd.read_csv('../input/trainopeniit/trains_openiit (2).csv')

In [ ]:
test = df.iloc[:100, :]
df.head()

In [ ]:
from  sklearn.metrics import *

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
i=0
mdel_performance = pd.DataFrame(columns=['regressor', 'rmse', 'r2_score', 'mae', 'mape'], index = range(len(est)))
for i in range(len(est)):
    model = est[i]
    try:
        print("gettin on :", i, model)
        m = model()
        tests[i] ,trains[i]=Stacking(model=m,n_fold=10, train=df,test=test,y=y)
        
        print("rmse:", np.sqrt(mean_squared_error(y, trains[i])), "  r2:", r2_score(y, trains[i]), "  mae:", (mean_absolute_error(y, trains[i])), " mape:", (mean_absolute_percentage_error(y, trains[i])))
        mdel_performance.iloc[i, 0] = str(model)
        mdel_performance.iloc[i, 1] = (np.sqrt(mean_squared_error(y, trains[i])))
        mdel_performance.iloc[i, 2] = r2_score(y, trains[i])
        mdel_performance.iloc[i, 3] = (mean_absolute_error(y, trains[i]))
        mdel_performance.iloc[i, 4] = (mean_absolute_percentage_error(y, trains[i]))
        
    except:
        print('sorry failed on :', model)
    i+=1


In [ ]:
# trains[1] = [0]*len(trains)
mdel_performance.to_csv('perfomance.csv', index=False)
FileLinks('.')
mdel_performance
FileLinks('.')


In [ ]:
a = trains.dropna(axis=1)
trains = a.copy()

In [ ]:
# np.sqrt(-1*cross_val_score(xg, trains, y,scoring="neg_mean_squared_error", cv=10)).mean()
for i in trains.columns:
    sc=StandardScaler()
    trains[i] = sc.fit_transform(np.array(trains[i]).reshape(-1, 1))

In [ ]:
def cross(model, data, y, n_folds=10):
    y_pred = np.array([0]*len(df))
    kf = KFold(n_splits=n_folds)
    ii=0
    for train_idx, test_idx in kf.split(data):
        train = data.iloc[train_idx, :]
        test = data.iloc[test_idx, :]
        ytn = y[train_idx]
        yts = y[test_idx]
        model.fit(train, ytn)
        y_pred[test_idx] = model.predict(test).astype(int)
        print(ii)
        ii+=1
    return y_pred

In [ ]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
# from  import Regressor
xgb = XGBRegressor()
lgm = LGBMRegressor()
cat = CatBoostRegressor(verbose=0)

In [ ]:
trains = a.copy()
trains.head()

In [ ]:
lg=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features=0.7500000000000001, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=11, min_samples_split=9,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=-1, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [ ]:
%%time
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import *
from sklearn.neighbors import KNeighborsRegressor



In [ ]:
lg4 = ExtraTreesRegressor(bootstrap=False, max_features=0.7500000000000001, min_samples_leaf=17, min_samples_split=17, n_estimators=100)


In [ ]:
lg5 = RandomForestRegressor(bootstrap=True, max_features=0.8, min_samples_leaf=13, min_samples_split=19, n_estimators=100, n_jobs=-1)


In [ ]:

lg6 = make_pipeline(
    MinMaxScaler(),
    ExtraTreesRegressor(bootstrap=False, max_features=0.3, min_samples_leaf=3, min_samples_split=17, n_estimators=100)
)


In [ ]:
lg7 = RandomForestRegressor(bootstrap=True, max_features=0.45, min_samples_leaf=19, min_samples_split=16, n_estimators=100)


In [ ]:
# os.listdir('../input/openhvb/')

In [ ]:
# trains = pd.read_csv('../input/openhvb/open.csv')

In [ ]:
trains.to_csv('open.csv', index=False)
FileLinks('.')

In [ ]:
########  following approach failed due to less no of datapoints

# df['y'] = y
# y_pred = np.array([0]*len(df))
# for i in (df['Number of Open Complaints'].unique()):
#     dat = df[df['Number of Open Complaints']==i].drop('Number of Open Complaints', axis=1)
#     yt = dat['y']
#     dat = dat.drop('y', axis=1)
    
#     kf = KFold(n_splits=10)
#     for train_idx, test_idx in kf.split(dat):
#         train = dat.iloc[train_idx, :]
#         test = dat.iloc[test_idx, :]
#         ytn = y[train_idx]
#         yts = y[test_idx]
#         lg.fit(train, ytn)
#         y_pred[test_idx] = lg.predict(test).astype(int)
#     print(i)
    
# df = df.drop('y', axis=1)
# print(np.sqrt(mean_squared_error(y, y_pred)), "     ", r2_score(y, y_pred))
# df['Number of Open Complaints'].value_counts()

In [ ]:
# y_pred.dtype

In [ ]:
# test_idx.dtype

In [ ]:
# y[7251]
df_copy = df.copy()
trains_copy =  trains.copy()
trains.to_csv('savs.csv', index=False)
FileLinks('.')

In [ ]:
## model wise performance report
ess = [lg4, lg5, lg6, lg7, xgb, lgm, cat]
mdel_performance = pd.DataFrame(columns=['regressor', 'rmse', 'r2_score', 'mae', 'mape'], index = range(len(ess)))

i=0
for model in ess:
    y_pred = cross(model, trains, y)
    
    print("rmse:", np.sqrt(mean_squared_error(y, y_pred)), "  r2:", r2_score(y, y_pred), "  mae:", (mean_absolute_error(y, y_pred)), " mape:", (mean_absolute_percentage_error(y, y_pred)))
    mdel_performance.iloc[i, 0] = str(model)
    mdel_performance.iloc[i, 1] = (np.sqrt(mean_squared_error(y, y_pred)))
    mdel_performance.iloc[i, 2] = r2_score(y, y_pred)
    mdel_performance.iloc[i, 3] = (mean_absolute_error(y, y_pred))
    mdel_performance.iloc[i, 4] = (mean_absolute_percentage_error(y, y_pred))
    i+=1


In [ ]:
mdel_performance

In [ ]:
import numpy as np
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
r1 = CatBoostRegressor(verbose=0)
xgb = XGBRegressor()
lgm = LGBMRegressor()
cat = CatBoostRegressor(verbose=0)
r2 = RandomForestRegressor(n_estimators=10, random_state=1)
# Low performing models were removed ('vnslcvjvbb', xgb), ('.ug.vnvbb', lgm), ('vnviygbb', cat)
er = VotingRegressor([ ('vnvf b', lg4), ('vnbxf', lg5), ('vnhvghg bjb', lg6), ('vnvbb', lg7), ])


In [ ]:
y_pred1 = cross(er, trains, y)

In [ ]:
print(r2_score(y, y_pred1))
print(mean_absolute_percentage_error(y, y_pred1))

In [ ]:
mdel_performance.to_csv('perfomance_with_stack.csv', index=False)
FileLinks('.')
mdel_performance
FileLinks('.')

In [ ]:
r2 = r2_score(y, y_pred1)
adj_r2 = 1-(1-r2)*(df.shape[0] - 1)/(df.shape[0] - df.shape[1] - 1)


In [ ]:
print(r2_score(y, y_pred))
print(mean_absolute_percentage_error(y, y_pred))
print(adj_r2)

In [ ]:
print("cros validated r2 score and mape is : ")
print(" r2_score :", r2_score(y, y_pred1))
print(" mape ", mean_absolute_percentage_error(y, y_pred1))
print("adjusted r2 :", adj_r2)